In [3]:
import numpy as np

from numpy import random

from os.path import dirname, join

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup, CustomJS, HoverTool
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

from bokeh.driving import count

from bokeh.models import Range1d

import time
import serial

useSerial = True

if useSerial == True:
    # Estalish a connection with the CD48 board
    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000, timeout = 1) # open serial port; port name will 
                                                            # have to be changed on different computers


# Define a class of objects called Counter. Each counter (0-7) will get a widget checkbox 
# for Port A through Port D   
class Counter(object):
    def __init__(self, portA, portB, portC, portD):
        self.portA = portA
        self.portB = portB
        self.portC = portC
        self.portD = portD
    

# Set a variable for common height of widgets
widgetHeight = 20   


# Create Label Widgets for each Counter Row. Ex) the 0th row will say "Counter 0:"
counter_labels = []

for i in range(8):
    counter_labels.append(PreText(text='Counter: ' + str(i), width = 80, height = widgetHeight))
    
# define width of port labels
portLabelWidth = 30

# Create Label widgets for each Port Column
portA = PreText(text='A: ', width = portLabelWidth, height = widgetHeight)
portB = PreText(text='B: ', width = portLabelWidth, height = widgetHeight)
portC = PreText(text='C: ', width = portLabelWidth, height = widgetHeight)
portD = PreText(text='D: ', width = portLabelWidth, height = widgetHeight)


# create some blank widgets just to "take up space" so the layout looks nicer
blank = PreText(text='', width = 80, height = widgetHeight)
blank2 = PreText(text='', width = 10, height = widgetHeight)
blank3 = PreText(text='', width = 20, height = widgetHeight)

# Create Set Button to send counter information to CD48 board
set_button = Button(
    label='Set Counters',
    disabled=False,
    button_type='success' # 'success', 'info', 'warning', 'danger' or ''
)



# define width of checkbox widgets
CheckboxWidth = 30

# Create the list counters. Each element of counters is an object of the class Counter.
# This means each element of counters will have 4 widget checkboxes, each referred to as portA - portD
# Later, you can call the "activeness"of say counter 4, port D checkbox in this way: 
# counters[4].portD.active   
# If counters[4].portD.active = [0], then this box is checked. 
# If counters[4].portD.active = [], then this box is not checked.

counters = []

# loop through 8 times to create 4 widget checkboxes (portA, portB, portC, portD) 
# for each of counters 0 through 7
for i in range(8):
    counters.append(Counter(CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight)))
    

# create a list to store board settings in 
boardSettings = []

# the first element of boardSettings will just be the title "Board Settings:"
boardSettings.append(PreText(text="""Board Settings:""", width = 30, height = widgetHeight))

# for the remaining 12 items, just append blank ' ' which can be changed later
for i in np.arange(12):
    boardSettings.append(PreText(text=""" """, width = 30, height = widgetHeight))


# Define a function that can be called later. This function sends the 'P' command to the board, 
# then determines whether a given counter is currently "watching" a given port. 
# Then set the checkbox values accordingly (blah.active = [0] --> True, blah.active = [] --> False), 
# checkboxes are booleans
def set_checkboxes():
    
    # get the board settings
    s.write('P'.encode())
    time.sleep(0.5)
    settings = s.readlines()
    
    # add the board settings to the list boardSettings
    for i in np.arange(len(settings)):
        boardSettings[i+1].text = str(settings[i])
    
    # loop through 8 times to glean port settings for each counter
    for i in range(8):
        # for debugging purposes
        #print('counter = ', i)
        #print(settings[i][2:3])
        #print(settings[i][3:4])
        #print(settings[i][4:5])
        #print(settings[i][5:6])
        
        # the board writes back in bytes, not strings, hence the b'0' or b'1'.
        if settings[i][2:3] == b'0':
            counters[i].portA.active = []
        else:
            counters[i].portA.active = [0]
        
        if settings[i][3:4] == b'0':
            counters[i].portB.active = []
        else:
            counters[i].portB.active = [0]

        if settings[i][4:5] == b'0':
            counters[i].portC.active = []
        else:
            counters[i].portC.active = [0]

        if settings[i][5:6] == b'0':
            counters[i].portD.active = []
        else:
            counters[i].portD.active = [0]

# run the set_checkboxes function to give the checkboxes proper 
# intial values that reflect the current board settings
set_checkboxes()



# Create info labels that can warn the user if an error is made (you cannot clear a counter)
infoLabels = []

# append blank text ' ' that can be changed later
for i in range(8):
    infoLabels.append(PreText(text='', width = 400, height = widgetHeight))

    
# Create one horizontal box for each counter, which includes the port checkboxes
counterBoxes = []

for i in range(8):
    counterBoxes.append(row([counter_labels[i], 
                               counters[i].portA, 
                               counters[i].portB, 
                               counters[i].portC, 
                               counters[i].portD, 
                               blank2,
                               infoLabels[i]]))
    


# create a horizontal box with the port column labels
portsBox = row([blank, portA, portB, portC, portD])

# display our row with button and info_sent label
buttonRow = row([blank3, set_button])


# create a column of text widgets to display the board settings
boardSettingsColumn = column([boardSettings[0],
                              boardSettings[1],
                              boardSettings[2],
                              boardSettings[3],
                              boardSettings[4],
                              boardSettings[5],
                              boardSettings[6],
                              boardSettings[7],
                              boardSettings[8],
                              boardSettings[9],
                              boardSettings[10],
                              boardSettings[11],
                              boardSettings[12]])

# create the layout of our interface including all the rows and columns above
counterInterface = row([column([portsBox, 
                          counterBoxes[0], 
                          counterBoxes[1], 
                          counterBoxes[2], 
                          counterBoxes[3],
                          counterBoxes[4],
                          counterBoxes[5],
                          counterBoxes[6],
                          counterBoxes[7],
                          buttonRow]), boardSettingsColumn])


# define a function that will be run every time the Set Counters button is clicked
def send_info():
    
    # loop through 8 times, one for each counter
    for i in range(8):
        # set the values A-D to 0
        A = 0
        B = 0
        C = 0
        D = 0
        # clear the warning label
        infoLabels[i].text = ''
        # check the checkbox values of portA - portD, and change respective values as needed
        if counters[i].portA.active == [0]:
            A = 1
        if counters[i].portB.active == [0]:
            B = 1
        if counters[i].portC.active == [0]:
            C = 1
        if counters[i].portD.active == [0]:
            D = 1
            
        # if the sum of A - D is at least 1 (aka at least one port box is checked/True)
        # then write the counter setting command to the board
        if A + B + C + D > 0:
            countingCommand = 'S'+ str(i) + str(int(A)) + str(int(B)) + str(int(C)) + str(int(D)) + '\n'
            s.write(countingCommand.encode())
        # else (if the sum of A - D is 0), warn the user that this is not permitted. Do not send command to board
        else:
            infoLabels[i].text = 'Counter ' + str(i) + ' must watch at least 1 port.'
    
    # run the set_checkboxes function to ensure that the values of the checkboxes 
    # reflect the current board settings
    set_checkboxes()


    
# call the send_info function when the button is clicked.
set_button.on_click(send_info)



# This portion of the code is for the Three Graphs


useSerial = True

# Set up the CD48 board
#if useSerial:
#    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000) # May need to change the port number
    #s.write("S01100\n".encode()) 

    
    
# Initialize the data lists, time is the same for all three plots
# use a Column Data Source, intially all lists are empty

# Convention:
# single counts for G (gate) will be stored on Counter 0
# single counts for T (transmitted) will be stored on Counter 1
# single counts for R (reflected) will be stored on Counter 2
# coincidence counts for GT will be stored on Counter 3
# coincidence counts for GR will be stored on Counter 4
# three-fold coincidence counts for GTR will be stored on Counter 5

source = ColumnDataSource(dict(
    timeList=[],
    singleCountsG=[], 
    singleCountsT=[], 
    singleCountsR=[],
    coincidenceCountsGT=[],
    coincidenceCountsGR=[],
    coincidenceCountsGTR=[],
    accidentalCoincidencesGT=[],
    accidentalCoincidencesGR=[],
    accidentalCoincidencesGTR=[],
    ratioCountsGT=[],
    ratioCountsGR=[],
    ratioCountsGTR=[],
    g2=[]
    ))


# just commenting these out for now, can add them back later

# Add Hover Tools
GHover = [
    ("Time", "@timeList"),
    ("Counts", "@singleCountsG")
]

THover = [
    ("Time", "@timeList"),
    ("Counts", "@singleCountsT")
]

RHover = [
    ("Time", "@timeList"),
    ("Counts", "@singleCountsR")
]

GTHover = [
    ("Time", "@timeList"),
    ("Total Counts", "@coincidenceCountsGT"),
    ("Accidental Counts", "@accidentalCoincidencesGT")
]

GRHover = [
    ("Time", "@timeList"),
    ("Total Counts", "@coincidenceCountsGR"),
    ("Accidental Counts", "@accidentalCoincidencesGR")
]

GTRHover = [
    ("Time", "@timeList"),
    ("Total Counts", "@coincidenceCountsGTR"),
    ("Accidental Counts", "@accidentalCoincidencesGTR")
]

g2Hover = [
    ("Time", "@timeList"),
    ("g2 value", "@g2")
]



# Create our 6 plots (3 singles, 3 coincidences)
singleCountsGPlot = figure(plot_height=260, plot_width=1000, title="Single Counts Gate",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GHover
                          )

singleCountsTPlot = figure(plot_height=260, plot_width=1000, title="Single Counts Transmitted",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = THover
                          )

singleCountsRPlot = figure(plot_height=260, plot_width=1000, title="Single Counts Reflected",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = RHover
                          )

coincidenceCountsGTPlot = figure(plot_height=260, plot_width=1000, 
                                 title="Coincidence Counts Gate and Transmitted",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GTHover
                                )

coincidenceCountsGRPlot = figure(plot_height=260, plot_width=1000, 
                                 title="Coincidence Counts Gate and Reflected",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GRHover
                                )

coincidenceCountsGTRPlot = figure(plot_height=260, plot_width=1000, 
                                 title="Three-fold Coincidence Counts",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = GTRHover
                                 )

# Hover tools for plots with multiple lines are added later 
ratioPlot = figure(plot_height=380, plot_width=1000, 
                                 title="Coincidences to Single Counts Ratio",
              tools="crosshair,pan,reset,save,wheel_zoom"
                                 ) 

g2Plot = figure(plot_height=380, plot_width=1000, 
                                 title="g2",
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips = g2Hover
                                 )




# Set up slider for user to choose counting time (100ms to 5s)
countingTimeSlider = Slider(start=0.1, end=5, value=1, step=0.1, title="Counting Time (s)", width = 200)
countingTime = countingTimeSlider.value # initialize the variable to store the counting interval


# Set up slider for user to choose how many points are displayed (10 to 100)
numberPointsSlider = Slider(start=10, end=100, value=20, step=5, title="Number of Points Displayed", 
                            width = 200)
numberPointsDisplayed = numberPointsSlider.value # initialize the variable to store the counting interval



# Make the plots display only the most recent "numberPointsDisplayed" points
# This way, the graph will scroll to show the newest points
# range_padding keeps the end points a certain distance from the edge of the graph
singleCountsGPlot.x_range.follow = "end"
singleCountsGPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
singleCountsGPlot.x_range.range_padding = 0.1
singleCountsGPlot.x_range.range_padding_units = 'absolute'
    
singleCountsTPlot.x_range.follow = "end"
singleCountsTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
singleCountsTPlot.x_range.range_padding = 0.1
singleCountsTPlot.x_range.range_padding_units = 'absolute'

singleCountsRPlot.x_range.follow = "end"
singleCountsRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
singleCountsRPlot.x_range.range_padding = 0.1
singleCountsRPlot.x_range.range_padding_units = 'absolute'
    
coincidenceCountsGTPlot.x_range.follow = "end"
coincidenceCountsGTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
coincidenceCountsGTPlot.x_range.range_padding = 0.1
coincidenceCountsGTPlot.x_range.range_padding_units = 'absolute'

coincidenceCountsGRPlot.x_range.follow = "end"
coincidenceCountsGRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
coincidenceCountsGRPlot.x_range.range_padding = 0.1
coincidenceCountsGRPlot.x_range.range_padding_units = 'absolute'

coincidenceCountsGTRPlot.x_range.follow = "end"
coincidenceCountsGTRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
coincidenceCountsGTRPlot.x_range.range_padding = 0.1
coincidenceCountsGTRPlot.x_range.range_padding_units = 'absolute'

ratioPlot.x_range.follow = "end"
ratioPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
ratioPlot.x_range.range_padding = 0.1
ratioPlot.x_range.range_padding_units = 'absolute'

g2Plot.x_range.follow = "end"
g2Plot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
g2Plot.x_range.range_padding = 0.1
g2Plot.x_range.range_padding_units = 'absolute'



# Make each point of each plot a circle
singleCountsGPlot.circle(x = 'timeList', y = 'singleCountsG', source = source, line_width=6, line_alpha=0.6,
                        line_color = 'red')
singleCountsTPlot.circle(x = 'timeList', y = 'singleCountsT', source = source, line_width=6, line_alpha=0.6,
                        line_color = 'green')
singleCountsRPlot.circle(x = 'timeList', y = 'singleCountsR', source = source, line_width=6, line_alpha=0.6,
                        line_color = 'blue')

coincidenceCountsGTPlot.circle(x = 'timeList', y = 'coincidenceCountsGT', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'red')
coincidenceCountsGRPlot.circle(x = 'timeList', y = 'coincidenceCountsGR', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'green')
coincidenceCountsGTRPlot.circle(x = 'timeList', y = 'coincidenceCountsGTR', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'blue')


# Create scatter plots for the ratio plot and add their associated hover tools
GTPoints = ratioPlot.circle(x = 'timeList', y = 'ratioCountsGT', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'red', legend_label="GT")
ratioPlot.add_tools(HoverTool(renderers=[GTPoints], tooltips=[("Time","@timeList"), 
                                ("Coincidence", "GT"), ("Ratio","@ratioCountsGT")], mode='mouse'))

GRPoints = ratioPlot.circle(x = 'timeList', y = 'ratioCountsGR', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'green', legend_label="GR")
ratioPlot.add_tools(HoverTool(renderers=[GRPoints], tooltips=[("Time","@timeList"), 
                                ("Coincidence", "GR"), ("Ratio","@ratioCountsGR")], mode='mouse'))

GTRPoints = ratioPlot.circle(x = 'timeList', y = 'ratioCountsGTR', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'blue', legend_label="GTR")
ratioPlot.add_tools(HoverTool(renderers=[GTRPoints], tooltips=[("Time","@timeList"), 
                                ("Coincidence", "GTR"), ("Ratio","@ratioCountsGTR")], mode='mouse'))


g2Plot.circle(x = 'timeList', y = 'g2', source = source, 
                               line_width=6, line_alpha=0.6, line_color = 'indigo')


# Connect the points of each plot with a line
singleCountsGPlot.line(x = 'timeList', y = 'singleCountsG', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'red')
singleCountsTPlot.line(x = 'timeList', y = 'singleCountsT', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'green')
singleCountsRPlot.line(x = 'timeList', y = 'singleCountsR', source = source, line_width=3, line_alpha=0.6,
                        line_color = 'blue')

coincidenceCountsGTPlot.line(x = 'timeList', y = 'coincidenceCountsGT', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'red')
coincidenceCountsGRPlot.line(x = 'timeList', y = 'coincidenceCountsGR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'green')
coincidenceCountsGTRPlot.line(x = 'timeList', y = 'coincidenceCountsGTR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'blue')

ratioPlot.line(x = 'timeList', y = 'ratioCountsGT', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'red', legend_label="GT")
ratioPlot.line(x = 'timeList', y = 'ratioCountsGR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'green', legend_label="GR")
ratioPlot.line(x = 'timeList', y = 'ratioCountsGTR', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'blue', legend_label="GTR")

# Specify the location of the legend
ratioPlot.legend.location = "top_left"

g2Plot.line(x = 'timeList', y = 'g2', source = source, 
                               line_width=3, line_alpha=0.6, line_color = 'indigo')



# Set y-axis labels for graphs
singleCountsGPlot.yaxis.axis_label = "Counts"
singleCountsTPlot.yaxis.axis_label = "Counts"
singleCountsRPlot.yaxis.axis_label = "Counts"

coincidenceCountsGTPlot.yaxis.axis_label = "Counts"
coincidenceCountsGRPlot.yaxis.axis_label = "Counts"
coincidenceCountsGTRPlot.yaxis.axis_label = "Counts"

ratioPlot.yaxis.axis_label = "Ratio"

g2Plot.yaxis.axis_label = "g2"


# Set x-axis labels for bottom-most graphs

#singleCountsGPlot.xaxis.axis_label = "Time (s)"
#singleCountsTPlot.xaxis.axis_label = "Time (s)"
singleCountsRPlot.xaxis.axis_label = "Time (s)"

#coincidenceCountsGTPlot.xaxis.axis_label = "Time (s)"
#coincidenceCountsGRPlot.xaxis.axis_label = "Time (s)"
coincidenceCountsGTRPlot.xaxis.axis_label = "Time (s)"

#ratioPlot.xaxis.axis_label = "Time (s)"

g2Plot.xaxis.axis_label = "Time (s)"



# Set up START/STOP button, to start and stop counting
startStopButtonGroup = RadioButtonGroup(labels=["START", "STOP"], active=1, width = 200)

# Set up the reset button, to clear the graph of all data
resetGraphButton = Button(label="Reset", width = 200)

# Set up the save datda button, to export the data to a csv file
saveDataButton = Button(label="Save Data", width = 200)

# Make a label reminding counter settings
counterSettings = PreText(text = ''' 
Gate --> Port A
Transmitted --> Port B
Reflected --> Port C

Counter Settings should be
Counter 0: A
Counter 1: B
Counter 2: C
Counter 3: AB
Counter 4: AC
Counter 5: ABC''', width = 200)



    

# keep a running count of time elapsed, start at 0. This will help inform x-axis values.
runningTime = 0

# keep a count, add one each time update_data is called. 
# use this to reset y-axis scaling periodically
resetYAxisCount = 0


# use these to keep track of the end of y_range for all four plots
yEndG = 1000

yEndT = 1000

yEndR = 1000

yEndGT = 1000

yEndGR = 1000

yEndGTR = 1000

yEndRatio = 1.01

yEndG2 = 3



# Set y_ranges for all plots
# if you do not do this, you will not be able to alter it later

singleCountsGPlot.y_range = Range1d(-100, yEndG)

singleCountsTPlot.y_range = Range1d(-100, yEndT)

singleCountsRPlot.y_range = Range1d(-100, yEndR)

coincidenceCountsGTPlot.y_range = Range1d(-100, yEndGT)

coincidenceCountsGRPlot.y_range = Range1d(-100, yEndGR)

coincidenceCountsGTRPlot.y_range = Range1d(-100, yEndGTR)

ratioPlot.y_range = Range1d(-0.01, yEndRatio)

g2Plot.y_range = Range1d(-0.1, yEndG2)


# define global lists for last 5 data points
# use 6 zeros so we can simply change list elements, not append things

last5G = np.zeros(6)
last5T = np.zeros(6)
last5R = np.zeros(6)
last5GT = np.zeros(6)
last5GR = np.zeros(6)
last5GTR = np.zeros(6)
last5Ratio = np.zeros(18)
last5G2 = np.zeros(6)

# Define a fuction to call when the current document (curdoc) has periodic callback on
# This function appends new data to our plots through the stream data command
def update_data():
    
    # retrieve the value of desired counting time
    countingTime = countingTimeSlider.value
    
    global resetYAxisCount
    
    # use the global variable
    global runningTime
    
    # add the counting time to our running time
    runningTime = runningTime + countingTime
    
    
    
    s.write("c\n".encode())
    serialData = s.readline()
    counterData = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]
    
    # accidental = N1*N2*deltaT / dwell time
    accidentalGT = (counterData[0]*counterData[1]*25*(10**(-9)))/(countingTime)
    accidentalGR = (counterData[0]*counterData[2]*25*(10**(-9)))/(countingTime)
    
    numerator = (counterData[3]*counterData[2] + counterData[4]*counterData[1])*25*(10**(-9))
    accidentalGTR = (numerator)/(countingTime)
    
    # adjusted = total - accidental
    #adjustedCoincidenceCounts = counterData[2] - calculatedAccidental
    
    # ratio = Nc / NG
    # avoid dividing by zero
    if counterData[0] > 0:
        ratioGT = counterData[3]/counterData[0]
        ratioGR = counterData[4]/counterData[0]
        ratioGTR = counterData[5]/counterData[0]
    else:
        #nan = float('nan')
        ratioGT = 0 # need to alert the user that this value is not a number
        ratioGR = 0
        ratioGTR = 0
        
    # g2 = NGTR * NG / (NGT * NGR)
    # avoid dividing by zero 
    if counterData[3]*counterData[4] > 0:
        g2result = (counterData[5] * counterData[0]) / (counterData[3] * counterData[4])
    else:
        #nan = float('nan')
        g2result = 0 # need to alert the user that this value is not a number
    
    
    new_data = dict(
        timeList=[runningTime],
        singleCountsG=[counterData[0]],
        singleCountsT=[counterData[1]],
        singleCountsR=[counterData[2]],
        coincidenceCountsGT=[counterData[3]],
        coincidenceCountsGR=[counterData[4]],
        coincidenceCountsGTR=[counterData[5]],
        accidentalCoincidencesGT = [accidentalGT],
        accidentalCoincidencesGR = [accidentalGR],
        accidentalCoincidencesGTR = [accidentalGTR],
        ratioCountsGT=[ratioGT],
        ratioCountsGR=[ratioGR],
        ratioCountsGTR=[ratioGTR],
        g2=[g2result]#,
        )
    
    # Stream the new data to the plots.
    source.stream(new_data)
    
    # use global variable
    global yEndG
    global yEndT
    global yEndR
    global yEndGT
    global yEndGR
    global yEndGTR
    global yEndRatio
    global yEndG2
    
    # use global last five points lists
    
    global last5G
    global last5T
    global last5R
    global last5GT
    global last5GR
    global last5GTR
    global last5Ratio
    global last5G2
    
    # Rescale y-axes as needed:
    
    
    # find the value of the most recent data point
    lastG = source.data['singleCountsG'][-1]
    lastT = source.data['singleCountsT'][-1]
    lastR = source.data['singleCountsR'][-1]
    lastGT = source.data['coincidenceCountsGT'][-1]
    lastGR = source.data['coincidenceCountsGR'][-1]
    lastGTR = source.data['coincidenceCountsGTR'][-1]
    lastRatioGT = source.data['ratioCountsGT'][-1]
    lastRatioGR = source.data['ratioCountsGR'][-1]
    lastRatioGTR = source.data['ratioCountsGTR'][-1]
    lastG2 = source.data['g2'][-1]
    
    
    # check for spikes outside of y_range on all four plots
    # if new point y value is outside of y_range, then change range
    
    if lastG > yEndG:   
        # reset the y-axis scaling to show this new high point
        yEndG = lastG*1.5
        singleCountsGPlot.y_range.update(start = -lastG*0.1, end = yEndG)
        
    if lastT > yEndT:   
        # reset the y-axis scaling to show this new high point
        yEndT = lastT*1.5
        singleCountsTPlot.y_range.update(start = -lastT*0.1, end = yEndT)
        
    if lastR > yEndR:   
        # reset the y-axis scaling to show this new high point
        yEndR = lastR*1.5
        singleCountsRPlot.y_range.update(start = -lastR*0.1, end = yEndR)
        
    if lastGT > yEndGT:   
        # reset the y-axis scaling to show this new high point
        yEndGT = lastGT*1.5
        coincidenceCountsGTPlot.y_range.update(start = -lastGT*0.1, end = yEndGT)
        
    if lastGR > yEndGR:   
        # reset the y-axis scaling to show this new high point
        yEndGR = lastGR*1.5
        coincidenceCountsGRPlot.y_range.update(start = -lastGR*0.1, end = yEndGR)
        
    if lastGTR > yEndGTR:   
        # reset the y-axis scaling to show this new high point
        yEndGTR = lastGTR*1.5
        coincidenceCountsGTRPlot.y_range.update(start = -lastGTR*0.1, end = yEndGTR)
        
    # if any ratio goes above y range, use max value to extend y range
    if max([lastRatioGT, lastRatioGR, lastRatioGTR]) > yEndRatio:   
        # reset the y-axis scaling to show this new high point
        yEndRatio = max([lastRatioGT, lastRatioGR, lastRatioGTR])*1.5
        ratioPlot.y_range.update(start = -yEndRatio*0.1, end = yEndRatio)
    
    
    if lastG2 > yEndG2:   
    # reset the y-axis scaling to show this new high point
        yEndG2 = lastG2*1.5
        g2Plot.y_range.update(start = -lastG2*0.1, end = yEndG2)
    
    # for the first five data points, make them the relevant elements of the relevant list
    if resetYAxisCount < 5:
        
        # make the most recent data point the nth element of last5 points list
        last5G[int(resetYAxisCount)] = lastG
        last5T[int(resetYAxisCount)] = lastT
        last5R[int(resetYAxisCount)] = lastR
        last5GT[int(resetYAxisCount)] = lastGT
        last5GR[int(resetYAxisCount)] = lastGR
        last5GTR[int(resetYAxisCount)] = lastGTR
        last5G2[int(resetYAxisCount)] = lastG2
        
        last5Ratio[int(resetYAxisCount)] = lastRatioGT
        last5Ratio[int(resetYAxisCount)+6] = lastRatioGR
        last5Ratio[int(resetYAxisCount)+12] = lastRatioGTR
        
    # else if resetYAxisCount >= 5
    else:
        
        # make the most recent point the 6th element of the list
        last5G[5] = lastG
        # shift every element of the list back one index
        # this leaves the sixth spot open for the next point to be added next time around
        # having the most recent point repeated in both index [4] and index [5] will not 
        # change max calculations
        for i in np.arange(5):
            last5G[i] = last5G[i+1]
            
        last5T[5] = lastT
        for i in np.arange(5):
            last5T[i] = last5T[i+1]
            
        last5R[5] = lastR
        for i in np.arange(5):
            last5R[i] = last5R[i+1]
            
        last5GT[5] = lastGT
        for i in np.arange(5):
            last5GT[i] = last5GT[i+1]
            
        last5GR[5] = lastGR
        for i in np.arange(5):
            last5GR[i] = last5GR[i+1]
            
        last5GTR[5] = lastGTR
        for i in np.arange(5):
            last5GTR[i] = last5GTR[i+1]
           
        
        last5G2[5] = lastG2
        for i in np.arange(5):
            last5G2[i] = last5G2[i+1]
            
            
        
        last5Ratio[5] = lastRatioGT
        last5Ratio[11] = lastRatioGR
        last5Ratio[17] = lastRatioGTR
        for i in np.arange(5):
            last5Ratio[i] = last5Ratio[i+1]
        for i in np.arange(5):
            last5Ratio[i+6] = last5Ratio[i+7]
        for i in np.arange(5):
            last5Ratio[i+12] = last5Ratio[i+13]
        
        
    
    # if reset y axis count is divisible by 5 with no remainder
    # aka do what is in the if statement once every 5 data points
    if resetYAxisCount % 5 == 0:
        
        #sourceDataText.text = ' '.join(map(str, max(source.data['singleCountsA'][-5:])))
        
        
        # find the max values of the last 5 data points
        recentMaxG = max(last5G)
        recentMaxT = max(last5T)
        recentMaxR = max(last5R)
        recentMaxGT = max(last5GT)
        recentMaxGR = max(last5GR)
        recentMaxGTR = max(last5GTR)
        recentMaxRatio = max(last5Ratio)
        recentMaxG2 = max(last5G2)
        
        
        # if the recent max value is greater than 0:
        if recentMaxG > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndG = recentMaxG*1.5
            singleCountsGPlot.y_range.update(start = -recentMaxG*0.1, end = yEndG)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndG = 1000
            singleCountsGPlot.y_range.update(start = -100, end = yEndG)
            
        # if the recent max value is greater than 0:
        if recentMaxT > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndT = recentMaxT*1.5
            singleCountsTPlot.y_range.update(start = -recentMaxT*0.1, end = yEndT)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndT = 1000
            singleCountsTPlot.y_range.update(start = -100, end = yEndT)
            
        # if the recent max value is greater than 0:
        if recentMaxR > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndR = recentMaxR*1.5
            singleCountsRPlot.y_range.update(start = -recentMaxR*0.1, end = yEndR)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndR = 1000
            singleCountsRPlot.y_range.update(start = -100, end = yEndR)
            
        # if the recent max value is greater than 0:
        if recentMaxGT > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndGT = recentMaxGT*1.5
            coincidenceCountsGTPlot.y_range.update(start = -recentMaxGT*0.1, end = yEndGT)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndGT = 1000
            coincidenceCountsGTPlot.y_range.update(start = -100, end = yEndGT)
            
        # if the recent max value is greater than 0:
        if recentMaxGR > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndGR = recentMaxGR*1.5
            coincidenceCountsGRPlot.y_range.update(start = -recentMaxGR*0.1, end = yEndGR)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndGR = 1000
            coincidenceCountsGRPlot.y_range.update(start = -100, end = yEndGR)
            
        # if the recent max value is greater than 0:
        if recentMaxGTR > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndGTR = recentMaxGTR*1.5
            coincidenceCountsGTRPlot.y_range.update(start = -recentMaxGTR*0.1, end = yEndGTR)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndGTR = 1000
            coincidenceCountsGTRPlot.y_range.update(start = -100, end = yEndGTR)
          
        
        
        
        # if the recent max value is greater than 0:
        if recentMaxRatio > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndRatio = recentMaxRatio*1.5
            ratioPlot.y_range.update(start = -recentMaxRatio*0.1, end = yEndRatio)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis to -10 to 10
            yEndRatio = 0.2
            ratioPlot.y_range.update(start = -0.05, end = yEndRatio)
        
    
        # if the recent max value is greater than 0:
        if recentMaxG2 > 0:
        
            # reset the y-axis scaling to show recent max plus 0.5 * recent max
            yEndG2 = recentMaxG2*1.5
            g2Plot.y_range.update(start = -recentMaxG2*0.1, end = yEndG2)
        
        # else if recent max is 0:
        else:
            
            # reset y-axis
            yEndG2 = 2
            g2Plot.y_range.update(start = -0.1, end = yEndG2)
    
    # add 1 to reset y-axis counter
    resetYAxisCount = resetYAxisCount + 1
    
    
    





# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None


# Define a function to be called when the START/STOP button changes value
def change_periodic_callback(new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # clear the counters
        s.write("c\n".encode())
        serialData = s.readline()
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        #global countingTime
        countingTime = countingTimeSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
    
    # else if the STOP button is on
    elif startStopButtonGroup.active == 1:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        

# Define a function to be called when the reset button is pressed.   
def reset_plot():
    
    # set the running time back to 0
    global runningTime
    runningTime = 0
    
    global resetYAxisCount
    resetYAxisCount = 0
    
    global last5G
    global last5T
    global last5R
    global last5GT
    global last5GR
    global last5GTR
    global last5Ratio
    global last5G2
    
    last5G = np.zeros(6)
    last5T = np.zeros(6)
    last5R = np.zeros(6)
    last5GT = np.zeros(6)
    last5GR = np.zeros(6)
    last5GTR = np.zeros(6)
    last5Ratio = np.zeros(18)    
    last5G2 = np.zeros(6)
    
    # empty all data lists in source
    source.data = {k: [] for k in source.data}
    

def change_callback_period(attr, old, new):
    # use global variable
    global callback_id
    
    global numberPointsDisplayed
    
    global countingTime
    
    countingTime = countingTimeSlider.value
    
    # if the START button is on
    if startStopButtonGroup.active == 0:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        
        # Add a periodic_callback, the callback interval is stored in the variable countingTime
        callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
        
    # How many points should be displayed on screen at a time. Can turn this into a user input widget
    numberPointsDisplayed = numberPointsSlider.value


    # Make the plots display only the most recent "numberPointsDisplayed" points
    # This way, the graph will scroll to show the newest points
    singleCountsGPlot.x_range.follow = "end"
    singleCountsGPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsTPlot.x_range.follow = "end"
    singleCountsTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsRPlot.x_range.follow = "end"
    singleCountsRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTPlot.x_range.follow = "end"
    coincidenceCountsGTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGRPlot.x_range.follow = "end"
    coincidenceCountsGRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTRPlot.x_range.follow = "end"
    coincidenceCountsGTRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
    
    g2Plot.x_range.follow = "end"
    g2Plot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    ratioPlot.x_range.follow = "end"
    ratioPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
 


        
        
def change_number_points(attr, old, new):
    
    global numberPointsDisplayed
    global countingTime
    
    # retrieve slider values
    numberPointsDisplayed = numberPointsSlider.value 
    countingTime = countingTimeSlider.value


    # Make the plots display only the most recent "numberPointsDisplayed" points
    # This way, the graph will scroll to show the newest points
    singleCountsGPlot.x_range.follow = "end"
    singleCountsGPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsTPlot.x_range.follow = "end"
    singleCountsTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    singleCountsRPlot.x_range.follow = "end"
    singleCountsRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTPlot.x_range.follow = "end"
    coincidenceCountsGTPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGRPlot.x_range.follow = "end"
    coincidenceCountsGRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime

    coincidenceCountsGTRPlot.x_range.follow = "end"
    coincidenceCountsGTRPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
    
    g2Plot.x_range.follow = "end"
    g2Plot.x_range.follow_interval = (numberPointsDisplayed)*countingTime    

    ratioPlot.x_range.follow = "end"
    ratioPlot.x_range.follow_interval = (numberPointsDisplayed)*countingTime
    
    


    

    
# When the START/STOP button is clicked, call the change_periodic_callback function
startStopButtonGroup.on_click(change_periodic_callback)


# When the reset button is clicked, call the reset_plot function
resetGraphButton.on_click(reset_plot)

countingTimeSlider.on_change('value', change_callback_period)

# When the save data button is clicked, export the data into a csv file
saveDataButton.js_on_click(CustomJS(args=dict(source=source),
                            code=open(join(dirname(__file__), "download.js")).read()))

# When number of points is changed, change follow interval
numberPointsSlider.on_change('value', change_number_points)





# Set up layouts and add to document

# create a column of user input widgets
userInputWidgets = column(startStopButtonGroup, 
                          countingTimeSlider,
                          numberPointsSlider,
                          resetGraphButton,
                          saveDataButton
                         )


# create a layout with the set counters interface from the first half of this code
setCountersLayout = layout([column([counterInterface, counterSettings])], sizing_mode='scale_both')

# create a column with our three plots
plotsColumn1 = column([coincidenceCountsGTPlot,
                       coincidenceCountsGRPlot,
                       coincidenceCountsGTRPlot
                      ])

plotsColumn2 = column([singleCountsGPlot, 
                       singleCountsTPlot,
                       singleCountsRPlot
                     ])

plotsColumn3 = column([ratioPlot, 
                       g2Plot
                     ])

# create a layout for the graphs interface, with the input widgets and the three plots
#displayCountsLayout = layout([[row([userInputWidgets, plotsColumn1, plotsColumn2, plotsColumn3])]] , 
#                             sizing_mode='scale_both')

# create a tab for the set counters interface
setCountersTab = Panel(child = setCountersLayout, 
                       title="Set Counters")



singlesCountsLayout = row([userInputWidgets, plotsColumn2])

coincidenceCountsLayout = row([userInputWidgets, plotsColumn1])

ratioG2Layout = row([userInputWidgets, plotsColumn3])

# create tabs for the graphs interface
singlesCountsTab = Panel(child = singlesCountsLayout, 
                         title="Single Counts")

coincidenceCountsTab = Panel(child = coincidenceCountsLayout, 
                         title="Coincidence Counts")

ratioG2Tab = Panel(child = ratioG2Layout, 
                         title="G2 and Ratios")

# combine all tabs into one layout
currentDocumentTabs = Tabs(tabs=[ coincidenceCountsTab, singlesCountsTab, ratioG2Tab, setCountersTab ])


# Add tabs to current document
curdoc().add_root(currentDocumentTabs)

curdoc().title = "G2 Alignment Interface"